In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np
#from scipy.misc import imread

import tensorflow.keras as keras
from tensorflow.keras import Model
from matplotlib.pyplot import imshow
from tensorflow.keras.callbacks import ModelCheckpoint
import cv2

from ultralytics import YOLO
import matplotlib.pyplot as plt
from PIL import Image

from sklearn.metrics import accuracy_score
import pandas as pd

In [3]:
data = keras.utils.image_dataset_from_directory(
    directory='../../data/Faces',
    labels='inferred',
    label_mode='categorical',
    batch_size=140,
    image_size=(1000, 1000),
)

Found 140 files belonging to 6 classes.


In [4]:
face_classifier = YOLO("../../app/src/face_detection/model/yolov8n-face.pt")

In [5]:
#convert to numpy array
numpy_generator = data.as_numpy_iterator()
X, y = numpy_generator.next()


In [6]:
X.nbytes

num_faces = 0
#create a dataset with cropped faces and its corresponding label
X_cropped = []
y_cropped = []
for i in range(140):
  #  print(i)
    face = np.array(X[i], dtype = 'uint8')
    faces = face_classifier.predict(face)
    boxes = faces[0].boxes.xyxy.tolist()
    if boxes:
        num_faces += 1
        #extract the face based on the output from the YOLOv8 model        
        left, bottom, right, top = boxes[0]
        cropped_face = face[int(bottom):int(top), int(left):int(right)]
        #resize cropped face to a std shape, 100x100 for now but can adjust this
        pil_face = Image.fromarray(np.uint8(cropped_face)).convert('L')
        pil_face = pil_face.resize((48, 48))
        numpy_cropped_face = np.array(pil_face)
        #append this to the new list containing all cropped faces
        X_cropped.append(numpy_cropped_face)
        corresponding_label = y[i]
        #append the labels to the new label list
        y_cropped.append(corresponding_label)
        #uncomment below lines of code to see the cropped face visualisation
        
del X

c:\Users\veena\OneDrive\Desktop\Nus\Y4S2\IS4242\Project Actual\moodify\emotion_detection\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0: 640x640 1 face, 164.9ms
Speed: 12.0ms preprocess, 164.9ms inference, 1052.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 95.1ms
Speed: 4.0ms preprocess, 95.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 88.2ms
Speed: 3.0ms preprocess, 88.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 84.7ms
Speed: 3.0ms preprocess, 84.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 87.8ms
Speed: 4.0ms preprocess, 87.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 108.6ms
Speed: 3.0ms preprocess, 108.6ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 90.0ms
Speed: 3.0ms preprocess, 90.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 85.5ms
Speed: 3.0ms preprocess, 85.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)



In [7]:
X_cropped = np.array(X_cropped)
y_cropped = np.array(y_cropped)
X_cropped.shape

(138, 48, 48)

In [8]:
model = keras.models.load_model('../../app/src/Mood_detector/trained_emotion.keras')

In [9]:
y_pred = model.predict(X_cropped)



sum(np.argmax(y_pred[:, [0,2,3,4,5,6]], axis = 1)==np.argmax(y_cropped, axis = 1))/len(y_pred)

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 261ms/step


0.42028985507246375

In [13]:
pd.Series(np.argmax(y_cropped[np.argmax(y_pred[:, [0,2,3,4,5,6]], axis = 1)==np.argmax(y_cropped, axis = 1)], axis = 1)).value_counts()

2    21
3    17
4     9
0     5
1     3
5     3
Name: count, dtype: int64